<a href="https://colab.research.google.com/github/garvjain7/Chatbot/blob/main/End_End_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step-by-step Flow

1. Upload a PDF file.
2. Load the document using a document loader.
3. Combine document pages into a single text.
4. Split the text into chunks.
5. Convert chunks into `Document` objects.
6. Generate embeddings and store them in a vector store (FAISS).
7. Perform similarity search and retrieval.
8. Pass retrieved context to an LLM using a prompt.
9. Build a simple chatbot on top of the retrieval pipeline.


## 1. Installing Required Libraries

In [ ]:
!pip install -q langchain-community langchain-google-genai faiss-cpu pymupdf

##2. File Upload (Google Colab)

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving cric.pdf to cric.pdf


In [ ]:
print(uploaded)

{'cric.pdf': b'%PDF-1.4\n%\xd3\xeb\xe9\xe1\n1 0 obj\n<</Title (Cricket Laws)\n/Creator (Mozilla/5.0 \\(X11; Linux x86_64\\) AppleWebKit/537.36 \\(KHTML, like Gecko\\) HeadlessChrome/137.0.0.0 Safari/537.36)\n/Producer (Skia/PDF m137)\n/CreationDate (D:20250801104248+00\'00\')\n/ModDate (D:20250801104248+00\'00\')>>\nendobj\n3 0 obj\n<</ca .0902\n/BM /Normal>>\nendobj\n4 0 obj\n<</ca 1\n/BM /Normal>>\nendobj\n10 0 obj\n<</CA 1\n/ca 1\n/LC 0\n/LJ 0\n/LW 1\n/ML 4\n/SA true\n/BM /Normal>>\nendobj\n13 0 obj\n<</ca .5\n/BM /Normal>>\nendobj\n14 0 obj\n<</Type /XObject\n/Subtype /Form\n/Resources <</ProcSet [/PDF /Text /ImageB /ImageC /ImageI]\n/ExtGState <</G4 4 0 R>>\n/Font <</F9 9 0 R>>>>\n/BBox [0 0 195 39]\n/Group <</Type /Group\n/S /Transparency\n/I true>>\n/Filter /FlateDecode\n/Length 143>> stream\nx\x9c=\x8bM\n\xc20\x14\x06\xf7\xef\x14\xdf\x05\xda~//I\x1b(Bc\x7f\xd6Jn\xa0(\x08]\xa8\xf7\x07\xa1\x01w\x033\xf3\x16k\xd5\x05\x10D\xa5F\xd5\xa11\xf5\x86\xdb.<\xccuC\x85\xcfS\xba\xcd\xe3\xf9\

In [ ]:
filename = list(uploaded.keys())[0]

##3. Document Loaders

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader


loader = PyMuPDFLoader(filename)

docs = loader.load()

In [ ]:
print(docs)

[Document(metadata={'producer': 'Skia/PDF m137', 'creator': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) HeadlessChrome/137.0.0.0 Safari/537.36', 'creationdate': '2025-08-01T10:42:48+00:00', 'source': 'cric.pdf', 'file_path': 'cric.pdf', 'total_pages': 4, 'format': 'PDF 1.4', 'title': 'Cricket Laws', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-08-01T10:42:48+00:00', 'trapped': '', 'modDate': "D:20250801104248+00'00'", 'creationDate': "D:20250801104248+00'00'", 'page': 0}, page_content='Cricket Laws\n🏏 1. Basics of the Game\nObjective: Score more runs than the opponent.\nTeams: Two teams of 11 players each.\nInnings: Each team gets one or two innings depending on the format.\nFormats:\nTest Cricket: Up to 5 days, unlimited overs.\nOne Day International (ODI): 50 overs per side.\nTwenty20 (T20): 20 overs per side.\n🧢 2. Player Roles\nBatsmen: Aim to score runs.\nBowlers: Aim to dismiss batsmen and restrict runs.\nFielders: Support bowlers by c

##4. Combining Document Content

In [ ]:
full_text = ""

for doc in docs:
  full_text += doc.page_content + "\n"

print(full_text)

##5. Text Splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 10
)

text_chunks = splitter.split_text(full_text)

print(text_chunks)
print(len(text_chunks))

['Cricket Laws\n🏏 1. Basics of the Game\nObjective: Score more runs than the opponent.\nTeams: Two teams of 11 players each.\nInnings: Each team gets one or two innings depending on the format.\nFormats:\nTest Cricket: Up to 5 days, unlimited overs.\nOne Day International (ODI): 50 overs per side.', 'Twenty20 (T20): 20 overs per side.\n🧢 2. Player Roles\nBatsmen: Aim to score runs.\nBowlers: Aim to dismiss batsmen and restrict runs.\nFielders: Support bowlers by catching the ball and stopping runs.\nWicketkeeper: Specialized fielder behind the stumps.\n⚖️ 3. Scoring Rules', 'Run: Completed when batsmen cross and reach opposite creases.\nFour (4 runs): Ball hits ground before boundary.\nSix (6 runs): Ball crosses boundary without touching the ground.\nExtras:\nWide ball: 1 run and extra delivery.\nNo ball: 1 run and extra delivery.\nCricket Laws\n1', '1\nBye: Runs taken without touching the bat.\nLeg bye: Runs taken after ball hits the batsman’s body.\n🚫 4. Dismissals (Ways to Get Out)\

##6. Converting Text Chunks into Document Objects

In [ ]:
from langchain_core.documents import Document

chunks = [Document(page_content=chunk) for chunk in text_chunks]

print(chunks)

[Document(metadata={}, page_content='Cricket Laws\n🏏 1. Basics of the Game\nObjective: Score more runs than the opponent.\nTeams: Two teams of 11 players each.\nInnings: Each team gets one or two innings depending on the format.\nFormats:\nTest Cricket: Up to 5 days, unlimited overs.\nOne Day International (ODI): 50 overs per side.'), Document(metadata={}, page_content='Twenty20 (T20): 20 overs per side.\n🧢 2. Player Roles\nBatsmen: Aim to score runs.\nBowlers: Aim to dismiss batsmen and restrict runs.\nFielders: Support bowlers by catching the ball and stopping runs.\nWicketkeeper: Specialized fielder behind the stumps.\n⚖️ 3. Scoring Rules'), Document(metadata={}, page_content='Run: Completed when batsmen cross and reach opposite creases.\nFour (4 runs): Ball hits ground before boundary.\nSix (6 runs): Ball crosses boundary without touching the ground.\nExtras:\nWide ball: 1 run and extra delivery.\nNo ball: 1 run and extra delivery.\nCricket Laws\n1'), Document(metadata={}, page_con

##7. Vector Stores

In [ ]:
from google.colab import userdata
gemini_key = userdata.get('GEMINI')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001",api_key = gemini_key,output_dimensionality=16)

vector_store = FAISS.from_documents(documents=chunks,embedding=embedding_model)



##8. Similarity Search (Testing Vector Store)



In [ ]:
results = vector_store.similarity_search("What is leg bye",k=2)

print(results)

[Document(id='69cddcf7-d1ad-4a6e-a0ef-567e92addea2', metadata={}, page_content='1\nBye: Runs taken without touching the bat.\nLeg bye: Runs taken after ball hits the batsman’s body.\n🚫 4. Dismissals (Ways to Get Out)\n1. Bowled\n2. Caught\n3. Leg Before Wicket (LBW)\n4. Run Out\n5. Stumped\n6. Hit Wicket\n7. Handled the Ball (obsolete in some formats)\n8. Obstructing the Field'), Document(id='939f6c7f-9f3e-42e3-a6b9-5cc5e81ba324', metadata={}, page_content='Run: Completed when batsmen cross and reach opposite creases.\nFour (4 runs): Ball hits ground before boundary.\nSix (6 runs): Ball crosses boundary without touching the ground.\nExtras:\nWide ball: 1 run and extra delivery.\nNo ball: 1 run and extra delivery.\nCricket Laws\n1')]


##9. Retriever

In [ ]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

In [ ]:
results = retriever.invoke("What is leg bye")

print(results)

[Document(id='69cddcf7-d1ad-4a6e-a0ef-567e92addea2', metadata={}, page_content='1\nBye: Runs taken without touching the bat.\nLeg bye: Runs taken after ball hits the batsman’s body.\n🚫 4. Dismissals (Ways to Get Out)\n1. Bowled\n2. Caught\n3. Leg Before Wicket (LBW)\n4. Run Out\n5. Stumped\n6. Hit Wicket\n7. Handled the Ball (obsolete in some formats)\n8. Obstructing the Field'), Document(id='939f6c7f-9f3e-42e3-a6b9-5cc5e81ba324', metadata={}, page_content='Run: Completed when batsmen cross and reach opposite creases.\nFour (4 runs): Ball hits ground before boundary.\nSix (6 runs): Ball crosses boundary without touching the ground.\nExtras:\nWide ball: 1 run and extra delivery.\nNo ball: 1 run and extra delivery.\nCricket Laws\n1'), Document(id='c33ea2ce-0f62-489e-8313-5f7250de1129', metadata={}, page_content='ODI: First 10 overs - only 2 fielders outside the 30-yard circle.\nT20: First 6 overs - same restriction.\n⏱️ 8. Timing and Match Duration\nTest: 5 days, 90 overs/day.\nODI: ~8 h

##10. Preparing Context for the LLM

In [ ]:
def get_context_string(documents):
  context_text= ""
  for doc in documents:
    context_text+= doc.page_content + "\n"

  return context_text

In [ ]:
context= get_context_string(results)

print(context)

1
Bye: Runs taken without touching the bat.
Leg bye: Runs taken after ball hits the batsman’s body.
🚫 4. Dismissals (Ways to Get Out)
1. Bowled
2. Caught
3. Leg Before Wicket (LBW)
4. Run Out
5. Stumped
6. Hit Wicket
7. Handled the Ball (obsolete in some formats)
8. Obstructing the Field
Run: Completed when batsmen cross and reach opposite creases.
Four (4 runs): Ball hits ground before boundary.
Six (6 runs): Ball crosses boundary without touching the ground.
Extras:
Wide ball: 1 run and extra delivery.
No ball: 1 run and extra delivery.
Cricket Laws
1
ODI: First 10 overs - only 2 fielders outside the 30-yard circle.
T20: First 6 overs - same restriction.
⏱️ 8. Timing and Match Duration
Test: 5 days, 90 overs/day.
ODI: ~8 hours with innings break.
T20: ~3 hours total.
🧾 9. Umpiring and Decisions



##11. LLM and Prompt Template

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

llm  = ChatGoogleGenerativeAI(model="gemini-2.5-flash",api_key = gemini_key)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a helpful cricket expert.
Answer ONLY using the following context from the uploaded PDF.
If the context is insufficient, just say "I don't know".

Context:
{context}

Question:
{question}
"""
)

In [ ]:
chain = prompt | llm

In [ ]:
query = "Who is Salman Khan"

retrieved_docs = retriever.invoke(query)


final_retrieved_docs = get_context_string(retrieved_docs)

print(final_retrieved_docs)


response = chain.invoke({
    "question": query,
    "context": final_retrieved_docs
})


print(response.content)

Twenty20 (T20): 20 overs per side.
🧢 2. Player Roles
Batsmen: Aim to score runs.
Bowlers: Aim to dismiss batsmen and restrict runs.
Fielders: Support bowlers by catching the ball and stopping runs.
Wicketkeeper: Specialized fielder behind the stumps.
⚖️ 3. Scoring Rules
1
Bye: Runs taken without touching the bat.
Leg bye: Runs taken after ball hits the batsman’s body.
🚫 4. Dismissals (Ways to Get Out)
1. Bowled
2. Caught
3. Leg Before Wicket (LBW)
4. Run Out
5. Stumped
6. Hit Wicket
7. Handled the Ball (obsolete in some formats)
8. Obstructing the Field
Cricket Laws
🏏 1. Basics of the Game
Objective: Score more runs than the opponent.
Teams: Two teams of 11 players each.
Innings: Each team gets one or two innings depending on the format.
Formats:
Test Cricket: Up to 5 days, unlimited overs.
One Day International (ODI): 50 overs per side.

I don't know.
